[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/kyama0321/gammachirpy/blob/main/gcfb_v211/demo_gammachirp.ipynb)

# demo_gammachirp_notebook
This notebook demonstrates:
- frequency response of gammachirp filterbank
- center frequency vs. ERB width
- filter level dependency
- input/output function of gammachirp finterbank
- excitation pattern with maskers 
- impulse renponse of gammachirp and gammatone
- frequency response of gammachirp and gammatone
   
with the GammachirPy packege.

Note:
- The original article is [HERE](https://doi.org/10.20697/jasj.66.10_506) (Irino, 2010, in Japanese)
- The original MATLAB code is [HERE](https://researchmap.jp/irino/資料公開/) (Irino, 2010-2020, in Japanese)

References:
- Irino, T. and Patterson, R.D.: JASA, Vol. 101, pp. 412-419, 1997.
- Irino, T. and Patterson, R.D.: JASA, Vol. 109, pp. 2008-2022, 2001.
- Patterson, R.D., Unoki, M. and Irino, T.: JASA, Vol. 114, pp. 1529-1542, 2003. 
- Irino, T. and Patterson, R.D.: IEEE Trans. ASLP, Vol. 14, pp. 2222-2232, 2006.

## Setup

### For Google Colaboratory Users
- remove comment-out and execute all scrips

In [ ]:
#!git clone https://github.com/kyama0321/gammachirpy

In [ ]:
#%cd "gammachirpy/gcfb_v211"

### For All Users

In [ ]:
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
%matplotlib inline

import utils as utils
import gcfb_v211 as gcfb
import gammachirp as gc

# color map
cmap = plt.get_cmap("tab10")

## Frequency Response

In [ ]:
# Parameter values from Patterson+ (2003)
n = 4       
b1 = 1.81   
c1 = -2.96  
b2 = 2.17   
c2 = 2.20   
frat0 = 0.466
frat1 = 0.0109

n_rsl = 2**12
fs = 44100
ps = 50 # input level

frat = frat0 + frat1*ps

In [ ]:
# peak frequency of each filter channel
fp_list = np.array([250, 500, 1000, 2000, 4000, 8000])
fp_xtick = np.array([100, 250, 500, 1000, 2000, 4000, 8000, 16000])
erb_n_xtick, _ = utils.freq2erb(fp_xtick)

In [ ]:
fig, ax = plt.subplots()

for cnt, fp in enumerate(fp_list):
    # calculate frequency renponse
    fr1, _ = gcfb.fp2_to_fr1(n, b1, c1, b2, c2, frat, fp)
    cgc_resp = gcfb.cmprs_gc_frsp(fr1, fs, n, b1, c1, frat, b2, c2, n_rsl)
    cgc_frsp = cgc_resp.cgc_frsp
    
    # convert level and frequency scale to plot
    cgc_frsp_db = 20 * np.log10(cgc_frsp/np.max(cgc_frsp))
    freq = np.array(cgc_resp.freq)
    erb_num, _ = utils.freq2erb(freq)

    # plot frequency renponse of each filter channel
    plt.plot(erb_num[0, :], cgc_frsp_db[0, :], color=cmap(cnt))
    ax.set_xlim([erb_n_xtick[0], erb_n_xtick[-1]])
    ax.set_ylim([-70, 5])
    ax.set_xticks(erb_n_xtick)
    ax.set_xticklabels(fp_xtick)
    ax.set_xlabel("frequency (Hz)")
    ax.set_ylabel("filter gain (dB)")
    plt.title("frequency response")

plt.grid()

plt.show()

## Center Frequency vs. ERB Width

In [ ]:
# convert linear frequency to ERB_N scale
freq = np.arange(100, 12000+1, 100)
_, erb_n = utils.freq2erb(freq)

# plot in log scale
fig, ax = plt.subplots(figsize=(5,5))
plt.loglog(freq, erb_n)
ax.set_xlim([50, 15000])
ax.set_ylim([20, 2500])
plt.grid(which='both')
ax.set_xlabel("center frequency (Hz)")
ax.set_ylabel("equivalent rectanglar bandwidth (Hz)")
plt.title("center frequency vs. ERB width")

plt.show()

## Filter Level Dependency

In [ ]:
marker = ['o','x','d','*','^','p','s']

fp = 2000 # peak frequency
ps_list = [30, 40, 50, 60, 70, 80, 90] # input level (dB)

In [ ]:
fig, ax = plt.subplots(figsize=(5,5))

cgc_frs_db_max = np.array([])
for cnt, ps in enumerate(ps_list):
    # input level changes dcGC parameters
    frat = frat0 + frat1*ps

    # calculate frequency renponse
    fr1, _ = gcfb.fp2_to_fr1(n, b1, c1, b2, c2, frat, fp) 
    cgc_resp = gcfb.cmprs_gc_frsp(fr1, fs, n, b1, c1, frat, b2, c2, n_rsl)
    cgc_frsp = cgc_resp.cgc_frsp

    # reference: peak level at the smallest input level (30 dB)
    if cnt == 0:
        cgc_frsp_ref = np.max(cgc_frsp)

    # calculate relative frequency response 
    cgc_frsp_db = 20 * np.log10(cgc_frsp/np.max(cgc_frsp_ref))
    freq = np.array(cgc_resp.freq)
    cgc_frs_db_max = np.append(cgc_frs_db_max, np.max(cgc_frsp_db))

    # plot
    plt.plot(freq[0, :], cgc_frsp_db[0, :], color=cmap(cnt))
    plt.plot(fp, cgc_frs_db_max[cnt], marker[cnt], color=cmap(cnt), markersize=8)
    ax.set_xlim([0, 4000])
    ax.set_ylim([-70, 5])
    ax.set_xticks([0, 1000, 2000, 3000, 4000])
    ax.set_xlabel('frequency (Hz)')
    ax.set_ylabel('filter gain (dB)')
    plt.grid()
    plt.text(fp*1.05, cgc_frs_db_max[cnt], f"{ps}")

plt.text(fp*1.15, 1.0, 'input level (dB)')
plt.title('frequency response')

plt.show()

## Input/Output Function

In [ ]:
fig, ax = plt.subplots(figsize=(5,5))

# adjust output level to 90 dB when the input level is 90 dB
output_level = cgc_frs_db_max + ps_list + 33

# plot I/O function with a "linear" I/O function
plt.plot(ps_list, output_level, 'k', ps_list, ps_list, ':k')

# plot markers at each corrspond I/O level 
for cnt in range(len(output_level)):
    plt.plot(ps_list[cnt], output_level[cnt], marker[cnt], \
        color=cmap(cnt), clip_on = False, markersize=8)

ax.set_xlim([30, 90])
ax.set_ylim([30, 90])
ax.set_xlabel("input level (dB)")
ax.set_ylabel("output level (dB)")
plt.title("I/O function")
plt.grid()

plt.show()

## Excitation Pattern with Maskers

In [ ]:
 # Param values from Patterson+ (2003)
n = 4
b1 = 1.81
c1 = -2.96
b2 = 2.17
c2 = 2.20
frat0 = 0.466
frat1 = 0.0109

n_rsl = 2**12
fs = 44100
num_ch = 100
freq_range = np.array([100, 6000])

erb_range, _ = utils.freq2erb(freq_range)
erb_p_list = np.linspace(erb_range[0], erb_range[1], num_ch)
fp_list, _ = utils.erb2freq(erb_p_list)

### Masker: 1-kHz sinusoidal signal

In [ ]:
# parameters
f_masker_list = np.array([1000])
ps_list = np.array([40, 60, 80])
peak_val_db = np.array([67.6, 73.4, 82.9])

fig = plt.subplots(figsize=(5,6))

# plot freq response at maximum sound pressure level
ax = plt.subplot(3, 1, 1)
ax.plot([f_masker_list/1000, f_masker_list/1000], [0, ps_list[-1]], color=cmap(len(ps_list)-1))
ax.set_xlim([0, fp_list[-1]/1000])
ax.set_ylim([40, 90])
ax.set_yticks([40, 60, 80])
ax.set_xlabel('frequency (kHz)')
ax.set_ylabel('sound\nlevel(dB)')
ax.grid()
plt.title('frequency response: 1-kHz sinusoidal signal')

# calc excitation pattern
for nps, ps in enumerate(ps_list): # sound pressure level
    frat = frat0 + frat1*ps
    ep = np.zeros(np.shape(fp_list))

    for nfp, fp in enumerate(fp_list): # peak frequency
        fr1, _ = gcfb.fp2_to_fr1(n, b1, c1, b2, c2, frat, fp)
        cgc_resp = gcfb.cmprs_gc_frsp(fr1, fs, n, b1, c1, frat, b2, c2, n_rsl)
        cgc_frsp = cgc_resp.cgc_frsp/np.max(np.abs(cgc_resp.cgc_frsp))

        freq = np.array(cgc_resp.freq)
        erb_n_num, _ = utils.freq2erb(freq)

        for nfm, f_masker in enumerate(f_masker_list):
            n_fat_masker = np.argmin(np.abs(freq - f_masker))
            ep[nfp] = ep[nfp] + cgc_frsp[0, n_fat_masker]**2

    # plot excitation pattern on ERB_N
    ax = plt.subplot(3, 1, 2)
    ep_db = 10*np.log10(ep) + peak_val_db[nps]
    ax.plot(erb_p_list, ep_db, color=cmap(nps))
    ax.set_xlim([erb_p_list[0], erb_p_list[-1]])
    ax.set_ylim([40, 90])
    ax.set_xticks(np.arange(5, 35, 5))
    ax.set_yticks([40, 60, 80])
    ax.set_xlabel('ERB_N number')
    ax.set_ylabel('excitation\n level (dB)')
    ax.grid()
    
    # text: sound pressure level (dB)
    n_ep_db = np.min(np.intersect1d(np.where(ep_db < ps/4+35), np.where(erb_p_list > 15)))
    ax.text(erb_p_list[n_ep_db], ep_db[n_ep_db]+2, f'{ps}')

    # plot excitation pattern on frequency, kHz
    ax = plt.subplot(3, 1, 3)
    ax.plot(fp_list/1000, ep_db)
    ax.set_xlim([0, fp_list[-1]/1000])
    ax.set_ylim([40, 90])
    ax.set_yticks([40, 60, 80])
    ax.set_xlabel('frequency (kHz)')
    ax.set_ylabel('excitation\n level (dB)')
    ax.text(fp_list[n_ep_db]/1000, ep_db[n_ep_db]+2, f'{ps}')
    ax.grid()

plt.tight_layout()
plt.show()

### Masker: complex tone

In [ ]:
# masker: tone
f_masker_list = np.arange(200, 2200, 200)
ps_list = np.array([70])
peak_val_db = np.array([77.08])

fig = plt.subplots(figsize=(5,6))

# plot freq response at maximum sound pressure level
ax = plt.subplot(3, 1, 1)
ax.plot([f_masker_list/1000, f_masker_list/1000], [0, ps_list[-1]], color=cmap(len(ps_list)-1))
ax.set_xlim([0, fp_list[-1]/1000])
ax.set_ylim([40, 90])
ax.set_yticks([40, 60, 80])
ax.set_xlabel('frequency (kHz)')
ax.set_ylabel('sound\nlevel(dB)')
ax.grid()
plt.title('frequency response: complex tone')

# calc excitation pattern
for nps, ps in enumerate(ps_list):
    frat = frat0 + frat1*ps
    ep = np.zeros(np.shape(fp_list))

    for nfp, fp in enumerate(fp_list):
        fr1, _ = gcfb.fp2_to_fr1(n, b1, c1, b2, c2, frat, fp)
        cgc_resp = gcfb.cmprs_gc_frsp(fr1, fs, n, b1, c1, frat, b2, c2, n_rsl)
        cgc_frsp = cgc_resp.cgc_frsp/np.max(np.abs(cgc_resp.cgc_frsp))

        freq = np.array(cgc_resp.freq)
        erb_n_num, _ = utils.freq2erb(freq)

        for nfm, f_masker in enumerate(f_masker_list):
            n_fat_masker = np.argmin(np.abs(freq - f_masker))
            ep[nfp] = ep[nfp] + cgc_frsp[0, n_fat_masker]**2

    # plot excitation pattern on ERB_N
    ax = plt.subplot(3, 1, 2)
    ep_db = 10*np.log10(ep) + peak_val_db[nps]
    ax.plot(erb_p_list, ep_db)
    ax.set_xlim([erb_p_list[0], erb_p_list[-1]])
    ax.set_ylim([40, 90])
    ax.set_yticks([40, 60, 80])
    ax.set_xticks(np.arange(5, 35, 5))
    ax.set_xlabel('ERB_N number')
    ax.set_ylabel('excitation\n level (dB)')
    ax.grid()
    
    # text: sound pressure level (dB)
    n_ep_db = np.min(np.intersect1d(np.where(ep_db < ps/4+35), np.where(erb_p_list > 15)))
    ax.text(erb_p_list[n_ep_db], ep_db[n_ep_db]+2, f'{ps}')

    # plot excitation pattern on frequency, kHz
    ax = plt.subplot(3, 1, 3)
    ax.plot(fp_list/1000, ep_db)
    ax.set_xlim([0, fp_list[-1]/1000])
    ax.set_ylim([40, 90])
    ax.set_yticks([40, 60, 80])
    ax.set_xlabel('frequency (kHz)')
    ax.set_ylabel('excitation\n level (dB)')
    ax.text(fp_list[n_ep_db]/1000, ep_db[n_ep_db]+2, f'{ps}')
    ax.grid()

plt.tight_layout()
plt.show()

### Impulse Response of Gammachirp & Gammatone 

In [ ]:
# auditory filter parameters
fp = 2000
fs = 44100
n = 4
b = 1.019 # default gammatone
cgt = 0
cgc = -3

# gammatone
gt_ir, len_gt, _, _ = gc.gammachirp(fp, fs, n, b, cgt)
gm_env, _, _, _ = gc.gammachirp(fp, fs, n, b, cgt, 0, 'env')

# gammachirp
gc_ir, len_gc, _, _ = gc.gammachirp(fp, fs, n, b, cgc)

In [ ]:
# plot
tpl = 8
npl = np.arange(tpl*fs/1000).astype(int)
tms = npl/fs*1000
bz = 2.5
gme = np.array([[1], [-1]]) * gm_env[0, [npl]]

fig, ax = plt.subplots()
plt.plot(tms, gc_ir[0, [npl]][0, :]+bz, label='Gammachirp')
plt.plot(tms, gt_ir[0, [npl]][0, :], '--', label='Gammatone')

plt.plot(tms, gme[0, :], ':k', tms, gme[1,:], ':k', \
         tms, gme[0, :]+bz,':k', tms, gme[1,:]+bz, ':k', \
         [0, tpl], [0, 0], 'k', \
         [0, tpl], [bz, bz], 'k')

ax.set_xlim([0, tpl])
ax.set_yticks(np.arange(-1, bz+1.5, 0.5))
ax.set_ylim([0-1.25, bz+1.25])
ax.set_yticklabels('')
ax.set_xlabel('time (ms)')
ax.set_ylabel('apmlitude (a.u.)')
ax.legend(loc='center right')
plt.title('impulse response')
plt.grid()

plt.show()

### Frequency Response of Gammachirp and Gammatone 

In [ ]:
# parameter settings
fs = 44100
n_frsl = 1024

# frequency renponse of each auditory filter
# gammatone
freq, frsp_gt = signal.freqz(gt_ir[0, :], 1, n_frsl, fs=fs)
gt_db = 20 * np.log10(np.abs(frsp_gt))
gt_db = gt_db  - np.max(gt_db)

# gammachirp
freq, frsp_gc = signal.freqz(gc_ir[0, :], 1, n_frsl, fs=fs)
gc_db = 20 * np.log10(np.abs(frsp_gc))
gc_db = gc_db  - np.max(gc_db)

# plot
fig, ax = plt.subplots()
plt.plot(freq, gc_db, label='Gammachirp')
plt.plot(freq, gt_db, '--', label='Gammatone')
ax.set_xlim([0, fp*2])
ax.set_ylim([-50, 5])
ax.set_xlabel('frequency (Hz)')
ax.set_ylabel('filter gain (dB)')
plt.title('frequency response')
plt.grid()
plt.legend()

plt.show()